### **1. Import Library**

In [3]:
import numpy as np
from scipy import stats
from scipy.stats import gaussian_kde
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
import warnings
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import json

# Mengatur agar angka desimal ditampilkan rapi (bukan notasi ilmiah)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# Mengabaikan warning dari geopandas agar output bersih
warnings.filterwarnings('ignore', 'UserWarning', module='geopandas')

print("Libraries diimport.")

Libraries diimport.


### **2. Path**

In [4]:
path_data_clean = "../data_clean/"
path_maps = "../maps/"
path_visualisasi = "../visualisasi_EDA/" # Folder untuk menyimpan hasil output

# Buat folder visualisasi klo belum ada
os.makedirs(path_visualisasi, exist_ok=True)

print(f"Data diambil dari: {path_data_clean}")
print(f"Peta diambil dari: {path_maps}")
print(f"Hasil EDA akan disimpan di: {path_visualisasi}")

Data diambil dari: ../data_clean/
Peta diambil dari: ../maps/
Hasil EDA akan disimpan di: ../visualisasi_EDA/


### **3. Load Data Bersih**

In [5]:
file_data_bersih = os.path.join(path_data_clean, "dataset_final_2021-2023.csv")

try:
    df = pd.read_csv(file_data_bersih)
    print(f"Berhasil memuat data: {file_data_bersih}")
    print(f"Dimensi data: {df.shape}")
    print(df.head())
except FileNotFoundError:
    print(f"ERROR: File {file_data_bersih} tidak ditemukan.")
    print("Jalankan '01_preprocessing.ipynb' terlebih dahulu!")
    raise

# Pastikan tipe data
cols_to_numeric = ['P1', 'UHH', 'HLS', 'RLS', 'Pengeluaran', 'TPT', 'Kepadatan']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Pisahkan hanya kolom numerik untuk analisis
df_numeric = df[cols_to_numeric]

Berhasil memuat data: ../data_clean/dataset_final_2021-2023.csv
Dimensi data: (105, 9)
                  Wilayah  Tahun   P1   UHH   HLS  RLS  Pengeluaran  TPT  \
0       Kabupaten Cilacap   2021 1.48 73.90 12.63 7.09        10534 9.97   
1      Kabupaten Banyumas   2021 2.35 73.80 13.03 7.63        11546 6.05   
2   Kabupaten Purbalingga   2021 2.10 73.21 12.00 7.25        10032 6.05   
3  Kabupaten Banjarnegara   2021 2.97 74.28 11.63 6.75         9407 5.86   
4       Kabupaten Kebumen   2021 3.24 73.55 13.35 7.55         9028 6.03   

   Kepadatan  
0        924  
1       1340  
2       1487  
3       1003  
4       1124  


### **4. Statistik Deskriptif**

In [6]:
print("="*60)
print("STATISTIK DESKRIPTIF VARIABEL PENELITIAN")
print("Provinsi Jawa Tengah (2021-2023)")
print("="*60)

numeric_cols = ['P1', 'UHH', 'HLS', 'RLS', 'Pengeluaran', 'TPT', 'Kepadatan']

# 1. Statistik Per Tahun
for year in [2021, 2022, 2023]:
    print(f"\n📅 TAHUN {year} (n = 35 Kabupaten/Kota)")
    print("-"*60)
    
    df_year = df[df['Tahun'] == year][numeric_cols]
    deskriptif = df_year.describe().T[['count', 'mean', 'std', 'min', 'max']]
    print(deskriptif.round(2))

# 2. Statistik Rata-rata 3 Tahun
print(f"\n{'='*60}")
print("📊 RINGKASAN: RATA-RATA 3 TAHUN (2021-2023)")
print("-"*60)
df_avg = df.groupby('Wilayah')[numeric_cols].mean()
deskriptif_avg = df_avg.describe().T[['count', 'mean', 'std', 'min', 'max']]
print(deskriptif_avg.round(2))

# 3. Tabel Perbandingan Mean Per Tahun
print(f"\n{'='*60}")
print("📈 PERBANDINGAN RATA-RATA ANTAR TAHUN")
print("-"*60)
comparison = df.groupby('Tahun')[numeric_cols].mean().T
comparison['Rata-rata'] = comparison.mean(axis=1)
print(comparison.round(2))

print(f"\n{'='*60}")

STATISTIK DESKRIPTIF VARIABEL PENELITIAN
Provinsi Jawa Tengah (2021-2023)

📅 TAHUN 2021 (n = 35 Kabupaten/Kota)
------------------------------------------------------------
             count     mean     std     min      max
P1           35.00     1.76    0.70    0.67     3.24
UHH          35.00    75.01    1.83   69.54    77.73
HLS          35.00    12.97    0.93   11.63    15.53
RLS          35.00     7.98    1.25    6.22    10.90
Pengeluaran  35.00 11139.20 1773.00 8573.00 15843.00
TPT          35.00     5.87    2.09    2.43     9.97
Kepadatan    35.00  2101.77 2421.80  491.00 11361.00

📅 TAHUN 2022 (n = 35 Kabupaten/Kota)
------------------------------------------------------------
             count     mean     std     min      max
P1           35.00     1.56    0.70    0.47     3.41
UHH          35.00    75.11    1.82   69.74    77.82
HLS          35.00    13.02    0.92   11.78    15.54
RLS          35.00     8.14    1.27    6.35    10.95
Pengeluaran  35.00 11533.60 1813.54 899

In [5]:
deskriptif = df_numeric.describe().T
print(deskriptif)

# Simpan ke CSV
deskriptif.to_csv(os.path.join(path_visualisasi, "statistik_deskriptif.csv"))
print("\n✅File tersimpan: statistik_deskriptif.csv")

             count     mean     std     min      25%      50%      75%  \
P1          105.00     1.62    0.69    0.47     1.03     1.57     1.95   
UHH         105.00    75.12    1.81   69.54    74.03    74.87    76.39   
HLS         105.00    13.01    0.91   11.63    12.44    12.89    13.35   
RLS         105.00     8.13    1.27    6.22     7.26     7.79     8.79   
Pengeluaran 105.00 11565.74 1814.00 8573.00 10277.00 11110.00 12522.00   
TPT         105.00     5.36    1.95    1.76     4.05     5.03     6.35   
Kepadatan   105.00  2113.94 2413.25  461.00   930.00  1180.00  1846.00   

                 max  
P1              3.41  
UHH            77.93  
HLS            15.55  
RLS            11.24  
Pengeluaran 16650.00  
TPT             9.97  
Kepadatan   11878.00  

✅File tersimpan: statistik_deskriptif.csv


### **5. Visualisasi Distribusi (Histogram)**

In [6]:
print("Membuat Histogram Distribusi Data...")

# Konfigurasi nama variabel
var_labels = {
    'P1': 'P1 - Indeks Kedalaman Kemiskinan (%)',
    'UHH': 'UHH - Usia Harapan Hidup (tahun)',
    'HLS': 'HLS - Harapan Lama Sekolah (tahun)',
    'RLS': 'RLS - Rata-rata Lama Sekolah (tahun)',
    'Pengeluaran': 'Pengeluaran per Kapita (ribu Rp)',
    'TPT': 'TPT - Tingkat Pengangguran Terbuka (%)',
    'Kepadatan': 'Kepadatan Penduduk (jiwa/km²)'
}

# Layout grid (3 kolom)
n_cols = 3
n_rows = math.ceil(len(df_numeric.columns) / n_cols)

colors = px.colors.qualitative.Bold

# Buat subplot
subplot_titles = [var_labels[col] for col in df_numeric.columns]

fig = make_subplots(
    rows=n_rows, 
    cols=n_cols, 
    subplot_titles=subplot_titles,
    vertical_spacing=0.13,
    horizontal_spacing=0.10
)

for i, column in enumerate(df_numeric.columns):
    row = (i // n_cols) + 1
    col = (i % n_cols) + 1
    
    color = colors[i % len(colors)]
    
    # Hitung statistik untuk ditampilkan
    data = df_numeric[column].dropna()
    mean_val = data.mean()
    median_val = data.median()
    std_val = data.std()
    
    # Tambahkan Histogram untuk variabel
    fig.add_trace(
        go.Histogram(
            x=df_numeric[column], 
            name=column,
            nbinsx=20,   
            marker_color=color,
            opacity=0.75,
            showlegend=False,
            histnorm='probability density',  # Normalisasi untuk overlay dengan kurva
            hovertemplate=(
                f'<b>{column}</b><br>' +
                '<b>Range</b>: %{x}<br>' +
                '<b>Density</b>: %{y:.4f}<br>' +
                '<extra></extra>'
            )
        ),
        row=row, col=col
    )
    
    # Hitung KDE
    kde = gaussian_kde(data)
    x_range = np.linspace(data.min(), data.max(), 200)
    y_kde = kde(x_range)
    
    # Tambahkan kurva KDE (mengikuti data asli)
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_kde,
            mode='lines',
            name='Kurva Density',
            line=dict(color='darkred', width=3, dash='solid'),
            showlegend=False,
            hovertemplate='<b>Kurva Density (KDE)</b><br>x: %{x:.2f}<br>density: %{y:.4f}<extra></extra>'
        ),
        row=row, col=col
    )
    
    # Tambahkan kurva distribusi normal teoretis untuk perbandingan
    x_range_norm = np.linspace(data.min(), data.max(), 200)
    y_norm = stats.norm.pdf(x_range_norm, mean_val, std_val)
    
    fig.add_trace(
        go.Scatter(
            x=x_range_norm,
            y=y_norm,
            mode='lines',
            name='Kurva Normal Teoretis',
            line=dict(color='black', width=2, dash='dash'),
            showlegend=False,
            hovertemplate='<b>Kurva Normal Teoretis</b><br>x: %{x:.2f}<br>density: %{y:.4f}<extra></extra>'
        ),
        row=row, col=col
    )

fig.update_layout(
    title={
        'text': "<b>Distribusi Variabel Penelitian</b>",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 22, 'family': 'Arial, sans-serif', 'color': '#2c3e50'}
    },
    height=300 * n_rows,
    bargap=0.1,
    showlegend=False,
    plot_bgcolor='rgba(240,240,240,0.5)',
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=11)
)

fig.update_annotations(font=dict(size=11, family="Arial, sans-serif", color="#34495e"))

for i, annotation in enumerate(fig['layout']['annotations']):
    if i < len(df_numeric.columns):
        annotation['font'] = dict(size=11, family="Arial, sans-serif", color="#2c3e50")
        annotation['y'] = annotation['y'] + 0.02  # Naikkan sedikit posisi judul

# Update axes untuk semua subplot
for i in range(1, len(df_numeric.columns) + 1):
    fig.update_xaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='lightgray',
        zeroline=False,
        row=((i-1) // n_cols) + 1, 
        col=((i-1) % n_cols) + 1
    )
    fig.update_yaxes(
        title_text="Density",
        showgrid=True, 
        gridwidth=1, 
        gridcolor='lightgray',
        zeroline=False,
        row=((i-1) // n_cols) + 1, 
        col=((i-1) % n_cols) + 1
    )

# Tambahkan anotasi legend manual di bagian bawah
fig.add_annotation(
    text="<b>Keterangan:</b> Garis hitam solid (—) menunjukkan kurva distribusi normal teoretis untuk perbandingan dengan distribusi data aktual",
    xref="paper", yref="paper",
    x=0.5, y=-0.04,
    showarrow=False,
    font=dict(size=10, color="gray"),
    xanchor='center'
)

# Tampilkan
fig.show()

# Simpan sebagai HTML
output_path = os.path.join(path_visualisasi, "histogram_distribusi.html")
fig.write_html(output_path)
print(f"✅ File tersimpan: {output_path}")

# Cetak statistik ringkas
print("\n📊 Ringkasan Statistik Distribusi:")
print("=" * 70)
for col in df_numeric.columns:
    data = df_numeric[col].dropna()
    print(f"{var_labels[col]}")
    print(f"  Mean: {data.mean():.2f} | Median: {data.median():.2f} | Std: {data.std():.2f}")
    print(f"  Range: {data.min():.2f} - {data.max():.2f}")
    print("-" * 70)

Membuat Histogram Distribusi Data...


✅ File tersimpan: ../visualisasi_EDA/histogram_distribusi.html

📊 Ringkasan Statistik Distribusi:
P1 - Indeks Kedalaman Kemiskinan (%)
  Mean: 1.62 | Median: 1.57 | Std: 0.69
  Range: 0.47 - 3.41
----------------------------------------------------------------------
UHH - Usia Harapan Hidup (tahun)
  Mean: 75.12 | Median: 74.87 | Std: 1.81
  Range: 69.54 - 77.93
----------------------------------------------------------------------
HLS - Harapan Lama Sekolah (tahun)
  Mean: 13.01 | Median: 12.89 | Std: 0.91
  Range: 11.63 - 15.55
----------------------------------------------------------------------
RLS - Rata-rata Lama Sekolah (tahun)
  Mean: 8.13 | Median: 7.79 | Std: 1.27
  Range: 6.22 - 11.24
----------------------------------------------------------------------
Pengeluaran per Kapita (ribu Rp)
  Mean: 11565.74 | Median: 11110.00 | Std: 1814.00
  Range: 8573.00 - 16650.00
----------------------------------------------------------------------
TPT - Tingkat Pengangguran Terbuka (%)
 

### **6. Matriks Korelasi**

In [7]:
print("Membuat Heatmap Korelasi...")

# Hitung matriks korelasi
corr_matrix = df_numeric.corr()

# Buat label yang lebih deskriptif untuk axis
var_labels_short = {
    'P1': 'P1<br>(Kemiskinan)',
    'UHH': 'UHH<br>(Usia Harapan)',
    'HLS': 'HLS<br>(Harapan Sekolah)',
    'RLS': 'RLS<br>(Lama Sekolah)',
    'Pengeluaran': 'Pengeluaran<br>(Per Kapita)',
    'TPT': 'TPT<br>(Pengangguran)',
    'Kepadatan': 'Kepadatan<br>Penduduk'
}

# Ganti index dan columns dengan label yang lebih deskriptif
corr_display = corr_matrix.copy()
corr_display.index = [var_labels_short[col] for col in corr_matrix.index]
corr_display.columns = [var_labels_short[col] for col in corr_matrix.columns]

# Buat heatmap dengan plotly
fig = px.imshow(
    corr_display,
    text_auto='.2f',
    aspect="auto",
    color_continuous_scale='RdBu_r', 
    zmin=-1, zmax=1,
    title="<b>Matriks Korelasi Antar Variabel</b><br><sub>Provinsi Jawa Tengah (2021-2023)</sub>",
    labels=dict(color="Koefisien")
)

# Update layout
fig.update_layout(
    title={
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#2c3e50'}
    },
    width=900,
    height=900,
    xaxis_title="",
    yaxis_title="",
    font=dict(family="Arial, sans-serif", size=11),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Update axes styling
fig.update_xaxes(side="bottom", tickangle=0)
fig.update_yaxes(tickangle=0)

# Update colorbar
fig.update_coloraxes(
    colorbar=dict(
        thickness=20,
        title='',
        len=0.9,
        tickvals=[-1, -0.5, 0, 0.5, 1],
        ticktext=['-1.0<br>(Negatif<br>Sempurna)', '-0.5', '0<br>(Tidak Ada)', '0.5', '1.0<br>(Positif<br>Sempurna)']
    )
)

# Tambahkan anotasi untuk interpretasi
fig.add_annotation(
    text="<b>Interpretasi:</b> Biru = Negatif | Putih = Tidak Ada | Merah = Positif",
    xref="paper", yref="paper",
    x=0.5, y=-0.08,
    showarrow=False,
    font=dict(size=10, color="gray"),
    xanchor='center'
)

# Tampilkan
fig.show()

# Simpan ke HTML
output_file = os.path.join(path_visualisasi, "heatmap.html")
fig.write_html(output_file)
print(f"✅ File tersimpan: {output_file}")

# Analisis Multikolinearitas
print("\n" + "="*70)
print("📊 ANALISIS MULTIKOLINEARITAS")
print("="*70)

# Deteksi korelasi tinggi (> 0.8)
pairs = corr_matrix.abs().unstack().sort_values(ascending=False)
high_corr_pairs = pairs[(pairs > 0.8) & (pairs < 1.0)]

if not high_corr_pairs.empty:
    print("\n⚠️  PERINGATAN: Ditemukan multikolinearitas!")
    print("-" * 70)
    
    # Tampilkan pasangan dengan korelasi tinggi
    displayed = set()
    for idx, value in high_corr_pairs.items():
        pair = tuple(sorted([idx[0], idx[1]]))
        if pair not in displayed:
            print(f"   • {idx[0]} ↔ {idx[1]}: r = {value:.3f}")
            displayed.add(pair)
    
    print("\n💡 Rekomendasi:")
    print("   - Pertimbangkan untuk menghapus salah satu variabel dari setiap pasangan")
    print("   - Atau gunakan teknik seperti PCA (Principal Component Analysis)")
    print("   - Atau lakukan analisis VIF (Variance Inflation Factor)")
else:
    print("\n✅ AMAN: Tidak ada multikolinearitas terdeteksi")
    print("   Semua korelasi antar variabel independen < 0.8")

# Tampilkan korelasi dengan variabel target (P1)
print("\n" + "="*70)
print("🎯 KORELASI DENGAN VARIABEL TARGET (P1 - Kemiskinan)")
print("="*70)

p1_corr = corr_matrix['P1'].drop('P1').sort_values(key=abs, ascending=False)

print("\nKorelasi diurutkan berdasarkan kekuatan (nilai absolut):\n")
for var, corr_val in p1_corr.items():
    # Tentukan arah dan kekuatan
    if abs(corr_val) >= 0.7:
        strength = "Sangat Kuat"
    elif abs(corr_val) >= 0.5:
        strength = "Kuat"
    elif abs(corr_val) >= 0.3:
        strength = "Sedang"
    else:
        strength = "Lemah"
    
    direction = "Positif" if corr_val > 0 else "Negatif"
    
    # Visualisasi bar sederhana
    bar_length = int(abs(corr_val) * 20)
    bar = '█' * bar_length
    
    print(f"  {var:12s}: {corr_val:+.3f}  {bar}  [{strength} {direction}]")

print("\n" + "="*70)
print("✨ Analisis korelasi selesai!")
print("="*70)

Membuat Heatmap Korelasi...


✅ File tersimpan: ../visualisasi_EDA/heatmap.html

📊 ANALISIS MULTIKOLINEARITAS

⚠️  PERINGATAN: Ditemukan multikolinearitas!
----------------------------------------------------------------------
   • HLS ↔ RLS: r = 0.909
   • Pengeluaran ↔ RLS: r = 0.821

💡 Rekomendasi:
   - Pertimbangkan untuk menghapus salah satu variabel dari setiap pasangan
   - Atau gunakan teknik seperti PCA (Principal Component Analysis)
   - Atau lakukan analisis VIF (Variance Inflation Factor)

🎯 KORELASI DENGAN VARIABEL TARGET (P1 - Kemiskinan)

Korelasi diurutkan berdasarkan kekuatan (nilai absolut):

  RLS         : -0.623  ████████████  [Kuat Negatif]
  UHH         : -0.596  ███████████  [Kuat Negatif]
  HLS         : -0.564  ███████████  [Kuat Negatif]
  Pengeluaran : -0.531  ██████████  [Kuat Negatif]
  Kepadatan   : -0.392  ███████  [Sedang Negatif]
  TPT         : +0.074  █  [Lemah Positif]

✨ Analisis korelasi selesai!


### **7. Menyiapkan Data Peta**

In [9]:
print("Menyiapkan data geospasial...")

numeric_cols = ['P1', 'UHH', 'HLS', 'RLS', 'Pengeluaran', 'TPT', 'Kepadatan']

# 1. Hitung Rata-rata 3 Tahun (2021-2023)
df_avg = df.groupby('Wilayah')[numeric_cols].mean().reset_index()

df_avg['wilayah_join'] = df_avg['Wilayah'].astype(str).str.replace(r'^\d+\s+', '', regex=True)
df_avg['wilayah_join'] = df_avg['wilayah_join'].str.replace('Kabupaten ', '').str.replace('Kota ', '').str.strip().str.upper()
print(f"Data BPS siap: {len(df_avg)} wilayah.")

# 2. Load Shapefile
filename_shp = "BATAS KABUPATEN KOTA DESEMBER 2019 DUKCAPIL.shp"
shapefile_path = os.path.join(path_maps, filename_shp)

try:
    gdf = gpd.read_file(shapefile_path)
    print(f"Shapefile berhasil dimuat. Total: {len(gdf)} wilayah (Indonesia).")
except Exception as e:
    print(f"ERROR: Gagal memuat {filename_shp}. {e}")
    raise

# 3. Standarisasi Nama di Peta
print("\nMelakukan pencocokan nama...")

def format_nama_peta(nama):
    nama = str(nama).upper().strip()
    
    # Jika sudah ada KAB. atau KABUPATEN
    if nama.startswith("KAB.") or nama.startswith("KAB "):
        nama = nama.replace("KAB.", "").replace("KAB ", "").strip()
        return "Kabupaten " + nama.title() # Jadikan "Kabupaten Cilacap"
        
    # Jika Kota
    elif nama.startswith("KOTA"):
        return nama.title() # Jadikan "Kota Semarang"
        
    # Jika cuma nama saja (misal "CILACAP"), anggap Kabupaten
    else:
        return "Kabupaten " + nama.title()

# Terapkan format ke kolom KAB_KOTA di Peta
gdf['Wilayah_Join'] = gdf['KAB_KOTA'].apply(format_nama_peta)

# Nama di Dataset sudah bersih ("Kabupaten ..."), jadi kita pakai langsung
df_avg['Wilayah_Join'] = df_avg['Wilayah']

# 4. Merge
gdf_map = gdf.merge(df_avg, on='Wilayah_Join', how='inner')

print(f"\n✅ Merge Selesai! Peta berhasil dipotong ke Jawa Tengah.")

# 5. Cek
gdf_final = gdf_map.dissolve(by='Wilayah_Join', aggfunc='first').reset_index()

print(f"\n✅ Jumlah wilayah final: {len(gdf_final)}")

# Cek Hasil
if len(gdf_final) == 35:
    print("🎉 SEMPURNA! Ada 35 Wilayah (Kabupaten & Kota terpisah).")
    print(gdf_final['Wilayah_Join'].head(35))
else:
    print(f"⚠️ Masih ada selisih ({len(gdf_final)} wilayah).")
    print("Cek nama yang mungkin tidak cocok/hilang:")
    # Cek bedanya dengan data asli
    hilang = set(df_avg['Wilayah_Join']) - set(gdf_final['Wilayah_Join'])
    print(hilang)

Menyiapkan data geospasial...
Data BPS siap: 35 wilayah.
Shapefile berhasil dimuat. Total: 515 wilayah (Indonesia).

Melakukan pencocokan nama...

✅ Merge Selesai! Peta berhasil dipotong ke Jawa Tengah.

✅ Jumlah wilayah final: 35
🎉 SEMPURNA! Ada 35 Wilayah (Kabupaten & Kota terpisah).
0     Kabupaten Banjarnegara
1         Kabupaten Banyumas
2           Kabupaten Batang
3            Kabupaten Blora
4         Kabupaten Boyolali
5           Kabupaten Brebes
6          Kabupaten Cilacap
7            Kabupaten Demak
8         Kabupaten Grobogan
9           Kabupaten Jepara
10     Kabupaten Karanganyar
11         Kabupaten Kebumen
12          Kabupaten Kendal
13          Kabupaten Klaten
14           Kabupaten Kudus
15        Kabupaten Magelang
16            Kabupaten Pati
17      Kabupaten Pekalongan
18        Kabupaten Pemalang
19     Kabupaten Purbalingga
20       Kabupaten Purworejo
21         Kabupaten Rembang
22        Kabupaten Semarang
23          Kabupaten Sragen
24       Kabupate

### **8. Membuat Peta Sebaran (Statis)**

In [ ]:
print("Memulai pembuatan peta...")

# 1. Konfigurasi Peta
map_config = {
    'P1': {
        'title': 'Indeks Kedalaman Kemiskinan',
        'cmap': 'OrRd',
        'format': '{:.2f}%',
        'desc': 'Semakin tinggi nilai, semakin dalam kemiskinan'
    },
    'UHH': {
        'title': 'Usia Harapan Hidup',
        'cmap': 'Greens',
        'format': '{:.2f} tahun',
        'desc': 'Semakin tinggi nilai, semakin baik kesehatan'
    },
    'HLS': {
        'title': 'Harapan Lama Sekolah',
        'cmap': 'Blues',
        'format': '{:.2f} tahun',
        'desc': 'Lama sekolah yang diharapkan akan ditempuh'
    },
    'RLS': {
        'title': 'Rata-rata Lama Sekolah',
        'cmap': 'Purples',
        'format': '{:.2f} tahun',
        'desc': 'Rata-rata lama sekolah penduduk 25+ tahun'
    },
    'Pengeluaran': {
        'title': 'Pengeluaran per Kapita',
        'cmap': 'YlGn',
        'format': 'Rp {:.0f}',
        'desc': 'Pengeluaran per kapita per bulan (ribu Rp)'
    },
    'TPT': {
        'title': 'Tingkat Pengangguran Terbuka',
        'cmap': 'Reds',
        'format': '{:.2f}%',
        'desc': 'Persentase angkatan kerja yang menganggur'
    },
    'Kepadatan': {
        'title': 'Kepadatan Penduduk',
        'cmap': 'YlOrBr',
        'format': '{:.0f} jiwa/km²',
        'desc': 'Jumlah penduduk per kilometer persegi'
    }
}

path_vis = "../visualisasi_EDA/" 
os.makedirs(path_vis, exist_ok=True)

for col, config in map_config.items():
    fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    
    # Hitung statistik untuk kolom ini
    data_valid = gdf_final[col].dropna()
    if len(data_valid) == 0:
        print(f"⚠️ Tidak ada data valid untuk {col}, skip...")
        plt.close()
        continue
    
    stats = {
        'mean': data_valid.mean(),
        'min': data_valid.min(),
        'max': data_valid.max(),
        'std': data_valid.std()
    }
    
    # Plot peta dengan warna
    gdf_final.plot(
        column=col,
        ax=ax,
        legend=True,
        cmap=config['cmap'],
        edgecolor='black',
        linewidth=0.5,
        missing_kwds={'color': 'lightgrey', 'edgecolor': 'black', 'label': 'Data Tidak Tersedia'},
        legend_kwds={
            'label': config['title'],
            'orientation': "vertical",
            'shrink': 0.8,
            'pad': 0.05
        }
    )
    
    # Tambahkan label nama wilayah untuk SEMUA 35 wilayah
    for idx, row in gdf_final.iterrows():
        if pd.notna(row.geometry):
            centroid = row.geometry.centroid
            
            wilayah_name = str(row['Wilayah'])
            
            wilayah_khusus = ['Tegal', 'Semarang', 'Pekalongan', 'Magelang']
            
            is_khusus = any(kota in wilayah_name for kota in wilayah_khusus)
            
            if is_khusus:
                if wilayah_name.startswith('Kabupaten '):
                    wilayah_name = wilayah_name.replace('Kabupaten ', 'Kab. ')
            else:
                wilayah_name = wilayah_name.replace('Kabupaten ', '').replace('Kota ', '')
            
            if len(wilayah_name) > 15:
                wilayah_name = wilayah_name[:13] + '.'
            
            ax.annotate(
                wilayah_name,
                xy=(centroid.x, centroid.y),
                fontsize=5.5,
                ha='center',
                va='center',
                fontweight='medium',
                color='black',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.6, edgecolor='none', linewidth=0)
            )
    
    # Judul utama
    ax.set_title(
        f"Peta Sebaran {config['title']}\nProvinsi Jawa Tengah (Rata-rata 2021-2023)",
        fontsize=16,
        fontweight='bold',
        pad=20
    )
    
    # Tambahkan informasi statistik di pojok kiri bawah
    stats_text = (
        f"Statistik:\n"
        f"Rata-rata: {config['format'].format(stats['mean'])}\n"
        f"Minimum: {config['format'].format(stats['min'])}\n"
        f"Maksimum: {config['format'].format(stats['max'])}\n"
        f"Std Dev: {config['format'].format(stats['std'])}"
    )
    
    ax.text(
        0.02, 0.02, stats_text,
        transform=ax.transAxes,
        fontsize=9,
        verticalalignment='bottom',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='black')
    )
    
    # Tambahkan deskripsi indikator di pojok kanan bawah
    ax.text(
        0.98, 0.02, config['desc'],
        transform=ax.transAxes,
        fontsize=8,
        verticalalignment='bottom',
        horizontalalignment='right',
        style='italic',
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7, edgecolor='gray')
    )
    
    # Tambahkan kompas Utara (North arrow)
    ax.text(
        0.95, 0.95, '↑ U',
        transform=ax.transAxes,
        fontsize=14,
        fontweight='bold',
        verticalalignment='top',
        horizontalalignment='center'
    )
    
    # Tambahkan grid dan koordinat
    ax.set_xlabel('Bujur Timur', fontsize=10)
    ax.set_ylabel('Lintang Selatan', fontsize=10)
    ax.tick_params(labelsize=8)
    
    # Tambahkan sumber data (posisi jauh ke bawah agar tidak menimpa xlabel)
    ax.text(
        0.5, -0.12, 'Sumber: BPS Jawa Tengah (2021-2023)',
        transform=ax.transAxes,
        fontsize=8,
        horizontalalignment='center',
        style='italic'
    )
    
    # Simpan Gambar PNG dengan kualitas tinggi
    file_png = os.path.join(path_vis, f"peta_{col}.png")
    plt.savefig(file_png, bbox_inches='tight', dpi=300, facecolor='white')
    plt.close()
    print(f"✅ [Statis] Tersimpan: {file_png}")

print("\n🎉 Semua peta berhasil dibuat!")
print(f"📂 Lokasi: {path_vis}")

Memulai pembuatan peta...
✅ [Statis] Tersimpan: ../visualisasi_EDA/peta_P1.png
✅ [Statis] Tersimpan: ../visualisasi_EDA/peta_UHH.png
✅ [Statis] Tersimpan: ../visualisasi_EDA/peta_HLS.png
✅ [Statis] Tersimpan: ../visualisasi_EDA/peta_RLS.png
✅ [Statis] Tersimpan: ../visualisasi_EDA/peta_Pengeluaran.png
✅ [Statis] Tersimpan: ../visualisasi_EDA/peta_TPT.png
✅ [Statis] Tersimpan: ../visualisasi_EDA/peta_Kepadatan.png

🎉 Semua peta berhasil dibuat!
📂 Lokasi: ../visualisasi_EDA/


In [13]:
print("Memulai pembuatan peta per tahun...")

# 1. Konfigurasi Peta
map_config = {
    'P1': {
        'title': 'Indeks Kedalaman Kemiskinan',
        'cmap': 'OrRd',
        'format': '{:.2f}%',
        'desc': 'Semakin tinggi nilai, semakin dalam kemiskinan'
    },
    'UHH': {
        'title': 'Usia Harapan Hidup',
        'cmap': 'Greens',
        'format': '{:.2f} tahun',
        'desc': 'Semakin tinggi nilai, semakin baik kesehatan'
    },
    'HLS': {
        'title': 'Harapan Lama Sekolah',
        'cmap': 'Blues',
        'format': '{:.2f} tahun',
        'desc': 'Lama sekolah yang diharapkan akan ditempuh'
    },
    'RLS': {
        'title': 'Rata-rata Lama Sekolah',
        'cmap': 'Purples',
        'format': '{:.2f} tahun',
        'desc': 'Rata-rata lama sekolah penduduk 25+ tahun'
    },
    'Pengeluaran': {
        'title': 'Pengeluaran per Kapita',
        'cmap': 'YlGn',
        'format': 'Rp {:.0f}',
        'desc': 'Pengeluaran per kapita per bulan (ribu Rp)'
    },
    'TPT': {
        'title': 'Tingkat Pengangguran Terbuka',
        'cmap': 'Reds',
        'format': '{:.2f}%',
        'desc': 'Persentase angkatan kerja yang menganggur'
    },
    'Kepadatan': {
        'title': 'Kepadatan Penduduk',
        'cmap': 'YlOrBr',
        'format': '{:.0f} jiwa/km²',
        'desc': 'Jumlah penduduk per kilometer persegi'
    }
}

path_vis = "../visualisasi_EDA/" 
os.makedirs(path_vis, exist_ok=True)

# 2. Fungsi format nama (pastikan sudah ada)
def format_nama_peta(nama):
    nama = str(nama).upper().strip()
    if nama.startswith("KAB.") or nama.startswith("KAB "):
        nama = nama.replace("KAB.", "").replace("KAB ", "").strip()
        return "Kabupaten " + nama.title()
    elif nama.startswith("KOTA"):
        return nama.title()
    else:
        return "Kabupaten " + nama.title()

# 3. Daftar tahun
years = [2021, 2022, 2023]

# 4. Loop untuk setiap tahun
for year in years:
    print(f"\n{'='*60}")
    print(f"📅 Membuat peta untuk TAHUN {year}...")
    print(f"{'='*60}")
    
    # Filter data untuk tahun ini
    df_year = df[df['Tahun'] == year].copy()
    
    # Standarisasi nama wilayah di data
    df_year['Wilayah_Join'] = df_year['Wilayah'].astype(str).str.replace(r'^\d+\s+', '', regex=True)
    
    # GUNAKAN gdf (shapefile asli), BUKAN gdf_final
    gdf_year = gdf.copy()
    gdf_year['Wilayah_Join'] = gdf_year['KAB_KOTA'].apply(format_nama_peta)
    
    # Merge data tahun ini ke peta
    gdf_year = gdf_year.merge(df_year, on='Wilayah_Join', how='inner')
    
    # Dissolve untuk menghilangkan duplikat geometry
    gdf_year = gdf_year.dissolve(by='Wilayah_Join', aggfunc='first').reset_index()
    
    print(f"✅ Data {year} siap: {len(gdf_year)} wilayah")
    
    # 5. Buat peta untuk setiap variabel
    for col, config in map_config.items():
        fig, ax = plt.subplots(1, 1, figsize=(14, 10))
        
        # Hitung statistik
        data_valid = gdf_year[col].dropna()
        if len(data_valid) == 0:
            print(f"⚠️ Tidak ada data untuk {col} tahun {year}, skip...")
            plt.close()
            continue
        
        stats = {
            'mean': data_valid.mean(),
            'min': data_valid.min(),
            'max': data_valid.max(),
            'std': data_valid.std()
        }
        
        # Plot peta
        gdf_year.plot(
            column=col,
            ax=ax,
            legend=True,
            cmap=config['cmap'],
            edgecolor='black',
            linewidth=0.5,
            missing_kwds={'color': 'lightgrey', 'edgecolor': 'black'},
            legend_kwds={
                'label': config['title'],
                'orientation': "vertical",
                'shrink': 0.8,
                'pad': 0.05
            }
        )
        
        # Label nama wilayah
        for idx, row in gdf_year.iterrows():
            if pd.notna(row.geometry):
                centroid = row.geometry.centroid
                wilayah_name = str(row['Wilayah'])
                
                wilayah_khusus = ['Tegal', 'Semarang', 'Pekalongan', 'Magelang']
                is_khusus = any(kota in wilayah_name for kota in wilayah_khusus)
                
                if is_khusus:
                    if wilayah_name.startswith('Kabupaten '):
                        wilayah_name = wilayah_name.replace('Kabupaten ', 'Kab. ')
                else:
                    wilayah_name = wilayah_name.replace('Kabupaten ', '').replace('Kota ', '')
                
                if len(wilayah_name) > 15:
                    wilayah_name = wilayah_name[:13] + '.'
                
                ax.annotate(
                    wilayah_name,
                    xy=(centroid.x, centroid.y),
                    fontsize=5.5,
                    ha='center',
                    va='center',
                    fontweight='medium',
                    color='black',
                    bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.6, edgecolor='none')
                )
        
        # Judul dengan TAHUN
        ax.set_title(
            f"Peta Sebaran {config['title']}\nProvinsi Jawa Tengah - Tahun {year}",
            fontsize=16,
            fontweight='bold',
            pad=20
        )
        
        # Box statistik
        stats_text = (
            f"Statistik {year}:\n"
            f"Rata-rata: {config['format'].format(stats['mean'])}\n"
            f"Minimum: {config['format'].format(stats['min'])}\n"
            f"Maksimum: {config['format'].format(stats['max'])}\n"
            f"Std Dev: {config['format'].format(stats['std'])}"
        )
        ax.text(0.02, 0.02, stats_text, transform=ax.transAxes, fontsize=9,
                verticalalignment='bottom',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='black'))
        
        # Deskripsi
        ax.text(0.98, 0.02, config['desc'], transform=ax.transAxes, fontsize=8,
                verticalalignment='bottom', horizontalalignment='right', style='italic',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7, edgecolor='gray'))
        
        # Kompas
        ax.text(0.95, 0.95, '↑ U', transform=ax.transAxes, fontsize=14,
                fontweight='bold', verticalalignment='top', horizontalalignment='center')
        
        ax.set_xlabel('Bujur Timur', fontsize=10)
        ax.set_ylabel('Lintang Selatan', fontsize=10)
        ax.tick_params(labelsize=8)
        
        # Sumber
        ax.text(0.5, -0.12, f'Sumber: BPS Jawa Tengah ({year})', transform=ax.transAxes,
                fontsize=8, horizontalalignment='center', style='italic')
        
        # SIMPAN DENGAN NAMA TAHUN
        file_png = os.path.join(path_vis, f"peta_{col}_{year}.png")
        plt.savefig(file_png, bbox_inches='tight', dpi=300, facecolor='white')
        plt.close()
        print(f"   ✅ peta_{col}_{year}.png")

print(f"\n{'='*60}")
print("🎉 SELESAI! Semua peta per tahun berhasil dibuat!")
print(f"📂 Lokasi: {path_vis}")
print(f"📊 Total: {len(years) * len(map_config)} file peta")
print(f"{'='*60}")

Memulai pembuatan peta per tahun...

📅 Membuat peta untuk TAHUN 2021...
✅ Data 2021 siap: 35 wilayah
   ✅ peta_P1_2021.png
   ✅ peta_UHH_2021.png
   ✅ peta_HLS_2021.png
   ✅ peta_RLS_2021.png
   ✅ peta_Pengeluaran_2021.png
   ✅ peta_TPT_2021.png
   ✅ peta_Kepadatan_2021.png

📅 Membuat peta untuk TAHUN 2022...
✅ Data 2022 siap: 35 wilayah
   ✅ peta_P1_2022.png
   ✅ peta_UHH_2022.png
   ✅ peta_HLS_2022.png
   ✅ peta_RLS_2022.png
   ✅ peta_Pengeluaran_2022.png
   ✅ peta_TPT_2022.png
   ✅ peta_Kepadatan_2022.png

📅 Membuat peta untuk TAHUN 2023...
✅ Data 2023 siap: 35 wilayah
   ✅ peta_P1_2023.png
   ✅ peta_UHH_2023.png
   ✅ peta_HLS_2023.png
   ✅ peta_RLS_2023.png
   ✅ peta_Pengeluaran_2023.png
   ✅ peta_TPT_2023.png
   ✅ peta_Kepadatan_2023.png

🎉 SELESAI! Semua peta per tahun berhasil dibuat!
📂 Lokasi: ../visualisasi_EDA/
📊 Total: 21 file peta


### **9. Dashboard Peta Interaktif**

In [ ]:
print("Memulai pembuatan peta interaktif...")

gdf_dashboard = gdf_final.to_crs("EPSG:4326")
gdf_dashboard = gdf_dashboard.set_index("Wilayah")
geojson_data = json.loads(gdf_dashboard.to_json())
    
vars_map = {
    'P1': 'Indeks Kedalaman Kemiskinan',
    'UHH': 'Usia Harapan Hidup',
    'HLS': 'Harapan Lama Sekolah',
    'RLS': 'Rata-rata Lama Sekolah',
    'Pengeluaran': 'Pengeluaran per Kapita',
    'TPT': 'Tingkat Pengangguran Terbuka',
    'Kepadatan': 'Kepadatan Penduduk'
}

# 3. Buat Figure (Wadah Peta)
fig = go.Figure()

# 4. Tambahkan "Layer" untuk SETIAP Variabel
first_var = True

for col, title in vars_map.items():
    fig.add_trace(
        go.Choropleth(
            geojson=geojson_data,
            locations=gdf_dashboard.index, # Kunci pencocokan (Nama Wilayah)
            z=gdf_dashboard[col],          # Data Angka
            featureidkey="id",             # ID di GeoJSON (karena kita set_index tadi, otomatis jadi id)
            colorscale="Viridis",          # Skala warna (bisa diganti: 'Reds', 'Blues', 'YlGnBu')
            name=title,
            visible=True if first_var else False, # Hanya yang pertama yang muncul awal
            colorbar_title=col
        )
    )
    first_var = False

# 5. Buat Tombol Dropdown
buttons = []
for i, (col, title) in enumerate(vars_map.items()):
    visibility = [False] * len(vars_map)
    visibility[i] = True # Nyalakan hanya layer ke-i
    
    button = dict(
        label=title, # Teks di menu dropdown
        method="update",
        args=[{"visible": visibility}, # Update visibilitas layer
                {"title": f"Peta Sebaran: {title} (2021-2023)"}] # Update judul peta
    )
    buttons.append(button)

# 6. Update Layout dengan Menu Dropdown
fig.update_layout(
    title_text=f"Dashboard Peta Jawa Tengah (Pilih Variabel)",
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.0, # Posisi menu (kiri)
        xanchor="left",
        y=1.1, # Posisi menu (atas)
        yanchor="top"
    )],
    geo=dict(
        fitbounds="locations", # Zoom otomatis ke area Jawa Tengah
        visible=False          # Sembunyikan peta dunia/latar belakang
    ),
    height=700,
    margin={"r":0,"t":50,"l":0,"b":0}
)

# 7. Simpan
output_file = os.path.join("../visualisasi_EDA/", "dashboard_peta_lengkap.html")
fig.write_html(output_file)
print(f"✅ DASHBOARD SIAP! File tersimpan di: {output_file}")  

In [ ]:
print("Memulai pembuatan peta interaktif...")

import json

# 1. Konversi ke CRS yang tepat dan siapkan data
gdf_dashboard = gdf_final.to_crs("EPSG:4326")
gdf_dashboard = gdf_dashboard.set_index("Wilayah")
geojson_data = json.loads(gdf_dashboard.to_json())

# 2. Konfigurasi variabel dengan detail lebih lengkap
vars_map = {
    'P1': {
        'title': 'Indeks Kedalaman Kemiskinan',
        'colorscale': 'OrRd',  # Merah untuk kemiskinan
        'unit': '%',
        'reversescale': False
    },
    'UHH': {
        'title': 'Usia Harapan Hidup',
        'colorscale': 'Greens',  # Hijau untuk kesehatan
        'unit': 'tahun',
        'reversescale': False
    },
    'HLS': {
        'title': 'Harapan Lama Sekolah',
        'colorscale': 'Blues',  # Biru untuk pendidikan
        'unit': 'tahun',
        'reversescale': False
    },
    'RLS': {
        'title': 'Rata-rata Lama Sekolah',
        'colorscale': 'Purples',  # Ungu untuk pendidikan
        'unit': 'tahun',
        'reversescale': False
    },
    'Pengeluaran': {
        'title': 'Pengeluaran per Kapita',
        'colorscale': 'YlGn',  # Kuning-hijau untuk ekonomi
        'unit': 'ribu Rp',
        'reversescale': False
    },
    'TPT': {
        'title': 'Tingkat Pengangguran Terbuka',
        'colorscale': 'Reds',  # Merah untuk pengangguran
        'unit': '%',
        'reversescale': False
    },
    'Kepadatan': {
        'title': 'Kepadatan Penduduk',
        'colorscale': 'YlOrBr',  # Kuning-orange untuk kepadatan
        'unit': 'jiwa/km²',
        'reversescale': False
    }
}

# 3. Buat Figure
fig = go.Figure()

# 4. Tambahkan layer untuk setiap variabel
first_var = True

for col, config in vars_map.items():
    data_col = gdf_dashboard[col]
    min_val = data_col.min()
    max_val = data_col.max()
    mean_val = data_col.mean()
    
    # Custom hover template
    hover_template = (
        "<b>%{location}</b><br><br>" +
        f"<b>{config['title']}</b><br>" +
        f"Nilai: %{{z:.2f}} {config['unit']}<br>" +
        f"<i>Rata-rata: {mean_val:.2f} {config['unit']}</i>" +
        "<extra></extra>"
    )
    
    fig.add_trace(
        go.Choropleth(
            geojson=geojson_data,
            locations=gdf_dashboard.index,
            z=gdf_dashboard[col],
            featureidkey="id",
            colorscale=config['colorscale'],
            reversescale=config['reversescale'],
            name=config['title'],
            visible=True if first_var else False,
            colorbar=dict(
                title=dict(
                    text=f"{col}<br>({config['unit']})",
                    side="right"
                ),
                thickness=15,
                len=0.7,
                x=1.02
            ),
            marker_line_color='white',
            marker_line_width=0.5,
            hovertemplate=hover_template
        )
    )
    first_var = False

# 5. Buat tombol dropdown
buttons = []
for i, (col, config) in enumerate(vars_map.items()):
    visibility = [False] * len(vars_map)
    visibility[i] = True
    
    # Hitung statistik untuk ditampilkan di judul
    data_col = gdf_dashboard[col]
    mean_val = data_col.mean()
    min_val = data_col.min()
    max_val = data_col.max()
    
    button = dict(
        label=config['title'],
        method="update",
        args=[
            {"visible": visibility},
            {
                "title": {
                    "text": (
                        f"<b>{config['title']}</b><br>" +
                        f"<sub>Provinsi Jawa Tengah (Rata-rata 2021-2023) | " +
                        f"Range: {min_val:.2f} - {max_val:.2f} {config['unit']}</sub>"
                    ),
                    "x": 0.5,
                    "xanchor": "center"
                }
            }
        ]
    )
    buttons.append(button)

# 6. Update layout
first_config = list(vars_map.values())[0]
first_col = list(vars_map.keys())[0]
first_data = gdf_dashboard[first_col]

fig.update_layout(
    title={
        "text": (
            f"<b>{first_config['title']}</b><br>" +
            f"<sub>Provinsi Jawa Tengah (Rata-rata 2021-2023) | " +
            f"Range: {first_data.min():.2f} - {first_data.max():.2f} {first_config['unit']}</sub>"
        ),
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 18, "family": "Arial, sans-serif"}
    },
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.02,
        xanchor="left",
        y=0.98,
        yanchor="top",
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="#ccc",
        borderwidth=1,
        font=dict(size=11)
    )],
    geo=dict(
        fitbounds="locations",
        visible=False,
        bgcolor='rgba(0,0,0,0)'
    ),
    height=750,
    margin={"r": 100, "t": 100, "l": 20, "b": 20},
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=11)
)

# 7. Tambahkan anotasi sumber data
fig.add_annotation(
    text="Sumber: BPS Jawa Tengah (2021-2023)",
    xref="paper", yref="paper",
    x=0.5, y=-0.02,
    showarrow=False,
    font=dict(size=9, color="gray", family="Arial, sans-serif"),
    xanchor='center'
)

# 8. Simpan
output_file = os.path.join("../visualisasi_EDA/", "dashboard_peta_interaktif.html")
fig.write_html(output_file)
print(f"✅ DASHBOARD INTERAKTIF SIAP!")
print(f"📂 File tersimpan di: {output_file}")

# 9. Cetak ringkasan
print("\nRingkasan Variabel di Dashboard:")
print("=" * 70)
for col, config in vars_map.items():
    data_col = gdf_dashboard[col]
    print(f"✓ {config['title']}: {data_col.min():.2f} - {data_col.max():.2f} {config['unit']}")
print("=" * 70)

Memulai pembuatan peta interaktif...
✅ DASHBOARD INTERAKTIF SIAP!
📂 File tersimpan di: ../visualisasi_EDA/dashboard_peta_interaktif.html

Ringkasan Variabel di Dashboard:
✓ Indeks Kedalaman Kemiskinan: 0.59 - 3.18 %
✓ Usia Harapan Hidup: 69.75 - 77.80 tahun
✓ Harapan Lama Sekolah: 11.75 - 15.54 tahun
✓ Rata-rata Lama Sekolah: 6.32 - 10.95 tahun
✓ Pengeluaran per Kapita: 9051.33 - 16281.33 ribu Rp
✓ Tingkat Pengangguran Terbuka: 2.10 - 9.44 %
✓ Kepadatan Penduduk: 482.33 - 11499.00 jiwa/km²


In [ ]:
import plotly.graph_objects as go
import json
import pandas as pd
import geopandas as gpd
import os

print("Membuat Dashboard Peta Interaktif (Variabel + Tahun)...")

print("Step 1: Menyiapkan data...")

# A. Load data bersih
df = pd.read_csv("../data_clean/dataset_final_2021-2023.csv")
numeric_cols = ['P1', 'UHH', 'HLS', 'RLS', 'Pengeluaran', 'TPT', 'Kepadatan']

# B. Hitung rata-rata 3 tahun
df_avg = df.groupby('Wilayah')[numeric_cols].mean().reset_index()

# C. Load shapefile
shapefile_path = "../maps/BATAS KABUPATEN KOTA DESEMBER 2019 DUKCAPIL.shp"
gdf = gpd.read_file(shapefile_path)

# D. Standarisasi nama wilayah untuk merge
df_avg['Wilayah_Join'] = (df_avg['Wilayah']
    .astype(str)
    .str.replace(r'^\d+\s+', '', regex=True)
    .str.replace('Kabupaten ', '')
    .str.replace('Kota ', '')
    .str.strip()
    .str.upper()
)

# E. Standarisasi nama di shapefile
target_col_shp = 'KAB_KOTA'

if target_col_shp in gdf.columns:
    gdf['Wilayah_Join'] = gdf[target_col_shp].astype(str).str.upper().str.strip()
    
    # F. Merge data ke peta
    gdf_final = gdf.merge(df_avg, on='Wilayah_Join', how='left')
    
    # Cek hasil merge
    unmatched = df_avg[~df_avg['Wilayah_Join'].isin(gdf['Wilayah_Join'])]
    if not unmatched.empty:
        print(f"⚠️ Warning: {len(unmatched)} wilayah tidak cocok:")
        print(unmatched['Wilayah_Join'].tolist())
    else:
        print(f"✅ Semua {len(df_avg)} wilayah berhasil di-merge ke peta")
else:
    print(f"❌ ERROR: Kolom '{target_col_shp}' tidak ada di shapefile!")
    print(f"Kolom tersedia: {gdf.columns.tolist()}")
    raise ValueError("Ganti variabel 'target_col_shp' dengan nama kolom yang benar!")

# --- 1. PERSIAPAN DATA UNTUK DASHBOARD ---
print("\nStep 2: Menyiapkan data dashboard...")

# A. Siapkan Data Rata-rata
gdf_dashboard = gdf_final.copy()
rename_dict = {col: f"{col}_Avg" for col in numeric_cols}
gdf_dashboard = gdf_dashboard.rename(columns=rename_dict)

# B. Siapkan Data Tahunan
df_pivot = df.pivot(index='Wilayah', columns='Tahun', values=numeric_cols)
df_pivot.columns = [f"{col}_{year}" for col, year in df_pivot.columns]
df_pivot = df_pivot.reset_index()

# C. Merge Data Tahunan ke Peta
gdf_dashboard = gdf_dashboard.merge(df_pivot, left_on='Wilayah_Join', right_on='Wilayah', how='left')

# D. Persiapan GeoJSON
gdf_dashboard = gdf_dashboard.to_crs("EPSG:4326")
gdf_dashboard = gdf_dashboard.set_index("Wilayah_Join")
geojson_data = json.loads(gdf_dashboard.to_json())

print(f"✅ Data siap: {len(gdf_dashboard)} wilayah dengan {len(gdf_dashboard.columns)} kolom")

# --- 2. KONFIGURASI VARIABEL ---
vars_config = {
    'P1': {
        'title': 'Indeks Kedalaman Kemiskinan',
        'scale': 'OrRd',
        'unit': '%',
        'desc': 'Semakin tinggi nilai, semakin dalam kemiskinan'
    },
    'UHH': {
        'title': 'Usia Harapan Hidup',
        'scale': 'Greens',
        'unit': 'tahun',
        'desc': 'Semakin tinggi nilai, semakin baik kesehatan'
    },
    'HLS': {
        'title': 'Harapan Lama Sekolah',
        'scale': 'Blues',
        'unit': 'tahun',
        'desc': 'Lama sekolah yang diharapkan akan ditempuh'
    },
    'RLS': {
        'title': 'Rata-rata Lama Sekolah',
        'scale': 'Purples',
        'unit': 'tahun',
        'desc': 'Rata-rata lama sekolah penduduk 25+ tahun'
    },
    'Pengeluaran': {
        'title': 'Pengeluaran per Kapita',
        'scale': 'YlGn',
        'unit': 'ribu Rp',
        'desc': 'Pengeluaran per kapita per bulan'
    },
    'TPT': {
        'title': 'Tingkat Pengangguran Terbuka',
        'scale': 'Reds',
        'unit': '%',
        'desc': 'Persentase angkatan kerja yang menganggur'
    },
    'Kepadatan': {
        'title': 'Kepadatan Penduduk',
        'scale': 'YlOrBr',
        'unit': 'jiwa/km²',
        'desc': 'Jumlah penduduk per kilometer persegi'
    }
}

years_list = ['Avg', 2021, 2022, 2023]
year_labels = {
    'Avg': 'Rata-rata 2021-2023',
    2021: 'Tahun 2021',
    2022: 'Tahun 2022',
    2023: 'Tahun 2023'
}

# --- 3. BUAT TRACES ---
fig = go.Figure()

first_var = True
for var_code, config in vars_config.items():
    col_name = f"{var_code}_Avg"
    
    # Cek apakah kolom ada
    if col_name not in gdf_dashboard.columns:
        print(f"⚠️ Warning: Kolom {col_name} tidak ditemukan, skip...")
        continue
    
    # Hitung statistik
    data_col = gdf_dashboard[col_name].dropna()
    if len(data_col) == 0:
        print(f"⚠️ Warning: {col_name} tidak memiliki data valid, skip...")
        continue
        
    mean_val = data_col.mean()
    min_val = data_col.min()
    max_val = data_col.max()
    
    # Hover template
    hover_txt = (
        "<b>%{location}</b><br><br>" +
        f"<b>{config['title']}</b><br>" +
        f"Nilai: <b>%{{z:.2f}}</b> {config['unit']}<br>" +
        f"<i>Rata-rata regional: {mean_val:.2f} {config['unit']}</i><br>" +
        f"<i>Range: {min_val:.2f} - {max_val:.2f}</i>" +
        "<extra></extra>"
    )

    fig.add_trace(
        go.Choropleth(
            geojson=geojson_data,
            locations=gdf_dashboard.index,
            z=gdf_dashboard[col_name],
            featureidkey="id",
            colorscale=config['scale'],
            name=config['title'],
            visible=True if first_var else False,
            colorbar=dict(
                title=dict(
                    text=f"<b>{var_code}</b><br>({config['unit']})",
                    side="right"
                ),
                thickness=15,
                len=0.7,
                x=1.02
            ),
            marker_line_color='white',
            marker_line_width=1,
            hovertemplate=hover_txt
        )
    )
    first_var = False

# --- 4. DROPDOWN MENU: VARIABEL ---
var_buttons = []
for i, (var_code, config) in enumerate(vars_config.items()):
    visibility = [False] * len(vars_config)
    visibility[i] = True
    
    # Hitung statistik untuk judul
    col_avg = f"{var_code}_Avg"
    if col_avg in gdf_dashboard.columns:
        data_col = gdf_dashboard[col_avg].dropna()
        mean_val = data_col.mean()
        range_txt = f"Range: {data_col.min():.2f} - {data_col.max():.2f} {config['unit']}"
    else:
        range_txt = ""
    
    button = dict(
        label=config['title'],
        method="update",
        args=[
            {"visible": visibility},
            {
                "title": {
                    "text": (
                        f"<b>{config['title']}</b><br>" +
                        f"<sub>{year_labels['Avg']} | {range_txt}</sub>"
                    ),
                    "x": 0.5,
                    "xanchor": "center"
                }
            }
        ]
    )
    var_buttons.append(button)

# --- 5. DROPDOWN MENU: TAHUN ---
year_buttons = []
for year in years_list:
    new_z_data = []
    new_hover_templates = []
    new_colorbars = []
    
    for var_code, config in vars_config.items():
        col_name = f"{var_code}_{year}"
        
        if col_name not in gdf_dashboard.columns:
            new_z_data.append([])
            new_hover_templates.append("")
            continue
        
        data_col = gdf_dashboard[col_name].dropna()
        if len(data_col) == 0:
            new_z_data.append([])
            new_hover_templates.append("")
            continue
            
        mean_val = data_col.mean()
        min_val = data_col.min()
        max_val = data_col.max()
        
        new_z_data.append(gdf_dashboard[col_name])
        
        # Hover template dengan info tahun
        txt = (
            "<b>%{location}</b><br><br>" +
            f"<b>{config['title']}</b><br>" +
            f"<i>{year_labels[year]}</i><br>" +
            f"Nilai: <b>%{{z:.2f}}</b> {config['unit']}<br>" +
            f"<i>Rata-rata: {mean_val:.2f} {config['unit']}</i><br>" +
            f"<i>Range: {min_val:.2f} - {max_val:.2f}</i>" +
            "<extra></extra>"
        )
        new_hover_templates.append(txt)

    button = dict(
        label=year_labels[year],
        method="restyle",
        args=[{"z": new_z_data, "hovertemplate": new_hover_templates}]
    )
    year_buttons.append(button)

# --- 6. LAYOUT FINAL ---
first_var_code = list(vars_config.keys())[0]
first_config = vars_config[first_var_code]
first_col = f"{first_var_code}_Avg"

if first_col in gdf_dashboard.columns:
    first_data = gdf_dashboard[first_col].dropna()
    range_text = f"Range: {first_data.min():.2f} - {first_data.max():.2f} {first_config['unit']}"
else:
    range_text = ""

fig.update_layout(
    title={
        "text": (
            f"<b>{first_config['title']}</b><br>" +
            f"<sub>{year_labels['Avg']} | {range_text}</sub>"
        ),
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 18, "family": "Arial, sans-serif", "color": "#2c3e50"}
    },
    geo=dict(
        fitbounds="locations",
        visible=False,
        bgcolor='rgba(0,0,0,0)'
    ),
    height=750,
    margin={"r": 120, "t": 140, "l": 20, "b": 40},
    paper_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=11),
    
    updatemenus=[
        # Menu 1: Variabel
        dict(
            buttons=var_buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.02,
            xanchor="left",
            y=1.12,
            yanchor="top",
            active=0,
            bgcolor="rgba(255, 255, 255, 0.95)",
            bordercolor="#ccc",
            borderwidth=1
        ),
        # Menu 2: Tahun
        dict(
            buttons=year_buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.35,
            xanchor="left",
            y=1.12,
            yanchor="top",
            active=0,
            bgcolor="rgba(255, 255, 255, 0.95)",
            bordercolor="#ccc",
            borderwidth=1
        )
    ]
)

# Label dropdown
fig.add_annotation(
    text="<b>📊 Pilih Variabel:</b>",
    x=0.02, y=1.16,
    xref="paper", yref="paper",
    showarrow=False,
    align="left",
    font=dict(size=12, family="Arial, sans-serif")
)
fig.add_annotation(
    text="<b>📅 Pilih Periode:</b>",
    x=0.35, y=1.16,
    xref="paper", yref="paper",
    showarrow=False,
    align="left",
    font=dict(size=12, family="Arial, sans-serif")
)

# Sumber data
fig.add_annotation(
    text="Sumber: BPS Jawa Tengah (2021-2023) | Dashboard Interaktif",
    xref="paper", yref="paper",
    x=0.5, y=-0.04,
    showarrow=False,
    font=dict(size=9, color="gray", family="Arial, sans-serif"),
    xanchor='center'
)

# --- 7. SIMPAN ---
output_file = os.path.join("../visualisasi_EDA/", "dashboard_peta_interaktif_lengkap.html")
fig.write_html(output_file)

print(f"\n{'='*70}")
print(f"✅ DASHBOARD INTERAKTIF BERHASIL DIBUAT!")
print(f"{'='*70}")
print(f"📂 File tersimpan di: {output_file}")
print(f"🎨 Fitur:")
print(f"   • Pilih Variabel: {len(vars_config)} indikator")
print(f"   • Pilih Periode: Rata-rata 2021-2023, atau per tahun")
print(f"   • Hover untuk info detail dan statistik")
print(f"   • Color scale sesuai konteks variabel")
print(f"   • Border putih antar wilayah")
print(f"{'='*70}\n")

Membuat Dashboard Peta Interaktif (Variabel + Tahun)...
Step 1: Menyiapkan data...
⚠️ Warning: 2 wilayah tidak cocok:
['SALATIGA', 'SURAKARTA']

Step 2: Menyiapkan data dashboard...
✅ Data siap: 519 wilayah dengan 32 kolom
